In [1]:
import bittensor
import torch
from tqdm import tqdm
from nuclei.gpt2 import GPT2Nucleus
from types import SimpleNamespace
from loguru import logger
bittensor.__debug_on__ = True
%load_ext autoreload

In [29]:
%autoreload 2

In [1]:

class Miner:
    def __init__( self, dataset: bittensor.Dataloader, endpoint: bittensor.Endpoint, child: bittensor.Endpoint ):
        
        # Local info.
        self.endpoint = endpoint
        
        # Child to call forward on.
        self.child = child
        
        # Text dataloader.
        self.dataset = dataset
        
        # Axon RPC server.
        # We attach the forward and backward passes to this miner class.
        # When this miner recieves a Forward/Backward request it calls these functions
        self.axon = bittensor.axon ( 
            local_port = self.endpoint.port,
            forward_callback = self.forward,
            backward_callback = self.backward
        )
        
        # Dendrite RPC Client.
        # Differentiable RPC function which calls Forward and Backward 
        # on passes endpoints.
        self.dendrite = bittensor.dendrite()
        
        # Torch NN Module with remote_forward and local_forwadd functions.
        # plus a routing function.
        self.nucleus = GPT2Nucleus( 
            routing_callback = self.route 
        )
        
        # Base Torch optimizer.
        self.optimizer = torch.optim.AdamW(self.nucleus.parameters(), lr = 0.01, betas = (0.9, 0.95) )
                
    # Function is called by the nucleus to query child and get responses.
    def route( self, inputs: torch.int64, query: torch.float32 ) -> torch.float32:
        
        # Is this a leaf node.
        if self.child == None:
            response = [torch.zeros( [inputs.shape[0], inputs.shape[1], bittensor.__network_dim__ ])]
        
        # Otherwise, makes differentiable calls.
        else:
            # Takes a list of endpoints and a list of inputs
            # Sends inputs to endpoints.
            responses, return_codes = self.dendrite.forward_text (
                endpoints = [self.child], 
                x = [inputs] 
            )
            
        return responses[0]
    
    # Function which is called when this miner recieves a forward request from a dendrite.
    def forward ( self, pubkey:str, inputs: torch.float32, modality:int ) -> torch.FloatTensor:
        # Call nucleus (locally, i.e. using the distillation model instead of calling the child)
        # return the last hidden layer.  
        output = self.nucleus.local_forward (
            inputs = inputs        
        )
        return output.local_hidden

    # Function which is called when this miner recieves a backward request. (Off for now.)
    def backward ( self, pubkey:str, inputs_x:torch.float32, grads_dy:torch.float32, modality:int ) -> torch.FloatTensor:
        return None
    
    # Start the axon serving endpoint.
    def start(self):
        self.axon.start()
        
    # Tear down the axon serving endpoint.
    def __del__(self):
        self.axon.stop()

    # Run a single epoch.
    def epoch(self):
        # ---- Next Batch ----
        for iteration, inputs in enumerate(self.dataset.dataloader( 100 )):     
            
            # ---- Forward pass ----
            output = self.nucleus.remote_forward(
                inputs = inputs,
                training = True,
            )

            # ---- Backward pass ----
            output.loss = output.local_target_loss + output.distillation_loss + output.remote_target_loss
            output.loss.backward() # Accumulates gradients on the nucleus.
            self.optimizer.step() # Applies accumulated gradients.
            self.optimizer.zero_grad() # Zeros out gradients for next accummulation

        


NameError: name 'bittensor' is not defined

In [31]:
# Dataset pulled from IPFS
dataset = bittensor.dataloader( max_corpus_size = 1000000 )

INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:149 - Retrieving a dataset file from the IPFS gateway...
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: magna_carta.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: meditations_marcus_aurelius.txt
INFO    |bittensor._dataloader.dataloader_impl:construct_text_corpus:168 - Added: rumi.txt


In [24]:
# Two fake bittensor endpoints for the miners.
endpoint_A = bittensor.endpoint( 
    uid = 0, 
    hotkey = '0', 
    ip = '0.0.0.0', 
    ip_type = 4, 
    port = 8080 , 
    modality = 0, 
    coldkey = 'N/A'
)
endpoint_B = bittensor.endpoint( 
    uid = 1, 
    hotkey = '1', 
    ip = '0.0.0.0', 
    ip_type = 4, 
    port = 8081, 
    modality = 0, 
    coldkey = 'N/A'  
)

In [28]:
# Create and start miner A
if "miner_A" in locals():
    del miner_A
    
miner_A = Miner( 
    dataset = dataset, 
    endpoint = endpoint_A, 
    child = endpoint_B 
)

miner_A.start()

None
SUCCESS |bittensor._axon.axon_impl:stop:492 - Axon has stopped serving on: 127.0.0.1:8080
SUCCESS |bittensor._axon.axon_impl:stop:492 - Axon has stopped serving on: 127.0.0.1:8080
SUCCESS |bittensor._axon.axon_impl:stop:492 - Axon has stopped serving on: 127.0.0.1:8080
SUCCESS |bittensor._axon.axon_impl:stop:492 - Axon has stopped serving on: 127.0.0.1:8080
ERROR   |bittensor._axon.axon_impl:start:478 - Forward and Backward callbacks must be subscribed on this axon before it starts. Got Forward = None and Backward = None


RuntimeError: Forward and Backward callbacks must be subscribed on this axon before it starts. Got Forward = None and Backward = None

In [26]:
# Create start miner B
if "miner_B" in locals():
    del miner_B
    
miner_B = Miner(
    dataset = dataset, 
    endpoint = endpoint_B, 
    child = None 
)
miner_B.start()

ERROR   |bittensor._axon.axon_impl:start:478 - Forward and Backward callbacks must be subscribed on this axon. Got None and None


RuntimeError: Forward and Backward callbacks must be subscribed on this axon. Got None and None

In [18]:
# Start training miner A.
miner_A.epoch()

Exception ignored in: <function Miner.__del__ at 0x13ba9ed30>
Traceback (most recent call last):
  File "<ipython-input-10-21b16cd8e160>", line 70, in __del__
AttributeError: 'Miner' object has no attribute 'axon'


SUCCESS |bittensor._axon.axon_impl:stop:492 - Axon has stopped serving on: 127.0.0.1:8080


TypeError: 'Miner' object is not callable